# `scsprox` Tutorial

`scsprox` creates fast proximal operators from CVXPY `Problem` objects.

For this tutorial, we first create a simple CVXPY problem.

In [1]:
import numpy as np
import cvxpy as cvx

m, n = 200, 100

np.random.seed(2)
A = np.random.randn(m,n)
b = np.random.randn(m)
x = cvx.Variable(n)

prob = cvx.Problem(cvx.Minimize(cvx.norm(A*x-b)))
prob.solve()

# the "true" solution, as computed by CVXPY
x_true = np.array(x.value).flatten()

# Form the `Prox` object

`scsprox` provides a single object, `Prox`.

We create a `Prox` object by passing in a CVXPY problem, `prob`, along with a dict, `prox_vars`, of the proximal variables.
That is, if `'x'` is a key in `prox_vars`, then we add the proximal regularization $\frac{\rho}{2}\|x-x_0 \|_2^2$
to the objective in `prob` to create the proximal problem.

During initialization, the `Prox` object forms a CySCS `Workspace`, which computes and stores the SCS factorization (which only needs to be computed once).

The `Prox` object accepts arbitrary CVXPY problems and any dict of related CVXPY variables to form the prox.

We set `verbose=True` below to confirm that SCS performs its matrix factorization.

In [2]:
from scsprox import Prox
prox_vars = {'x': x}
prox = Prox(prob, prox_vars, verbose=True)

----------------------------------------------------------------------------
	SCS v1.2.6 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012-2016
----------------------------------------------------------------------------
Lin-sys: sparse-direct, nnz in A = 20103
eps = 1.00e-03, alpha = 1.50, max_iters = 2500, normalize = 1, scale = 1.00
Variables n = 102, constraints m = 303
Cones:	soc vars: 303, soc blks: 2
Setup time: 6.73e-03s


# Evaluate the prox

Below, we'll evaluate the prox using the `Prox.do()` method on the input dict `x0` with `rho=1.0`.
`x0` is a dictionary of variable names and values (matching the names and variable sizes in `prox_vars`).

Note that SCS **doesn't** initialize, because the factorization has been cached,
and that this first call to `Prox.do()` takes 40 iterations.

Again, we make sure `verbose=True` to see the SCS status output.

In [3]:
x0 = {'x': np.zeros(n)}
rho = 1.0
x1 = prox.do(x0, rho, verbose=True)

SCS using variable warm-starting
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 1.76e+00  5.93e+00  9.86e-01 -2.12e+01  5.03e+01  0.00e+00  1.88e-03 
    40| 2.40e-07  1.84e-06  3.31e-08  1.11e+01  1.11e+01  2.21e-15  6.98e-03 
----------------------------------------------------------------------------
Status: Solved
Timing: Solve time: 7.00e-03s
	Lin-sys: nnz in L factor: 30408, avg solve time: 1.16e-04s
	Cones: avg projection time: 7.27e-07s
----------------------------------------------------------------------------
Error metrics:
dist(s, K) = 0.0000e+00, dist(y, K*) = 5.5511e-17, s'y/|s||y| = 3.8269e-16
|Ax + s - b|_2 / (1 + |b|_2) = 2.3961e-07
|A'y + c|_2 / (1 + |c|_2) = 1.8422e-06
|c'x + b'y| / (1 + |c'x| + |b'y|) = 3.3080e-08
----------------------------------------------------------------

# Prox status

We can see a `dict` of `Prox` status information with `Prox.info`:

In [4]:
prox.info

{'iter': 40,
 'setup_time': 0.006726358,
 'solve_time': 0.006997787,
 'status': 'Solved'}

# Automatic warm-starting

If we call `prox.do()` again, we can take advantage of warm-starting.
With the same `x0` and `rho` values, we see that the prox completes in 0 SCS iterations!

This is because the SCS solution from the previous `Prox.do()` call is used to warm-start this call.
Since `x0` and `rho` are the same, the SCS problem is identical, and no further iterations are needed.

In [5]:
x0 = {'x': np.zeros(n)}
rho = 1.0
x1 = prox.do(x0, rho, verbose=True)

SCS using variable warm-starting
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 1.58e-07  1.53e-06  1.49e-07  1.11e+01  1.11e+01  0.00e+00  2.09e-03 
----------------------------------------------------------------------------
Status: Solved
Timing: Solve time: 2.11e-03s
	Lin-sys: nnz in L factor: 30408, avg solve time: 2.79e-04s
	Cones: avg projection time: 3.52e-06s
----------------------------------------------------------------------------
Error metrics:
dist(s, K) = 2.6645e-15, dist(y, K*) = 2.2204e-16, s'y/|s||y| = -7.8502e-16
|Ax + s - b|_2 / (1 + |b|_2) = 1.5790e-07
|A'y + c|_2 / (1 + |c|_2) = 1.5322e-06
|c'x + b'y| / (1 + |c'x| + |b'y|) = 1.4893e-07
----------------------------------------------------------------------------
c'x = 11.0761, -b'y = 11.0760


# More realistic warm-starting

Of course, we usually won't try to compute the prox on exactly the same value, but instead, a slight perturbation of that value. Warm-starting still helps in this case, and still works automatically.

To see this, we call `Prox.do()` on `x1`, the output of the first prox computation.
SCS is warm-started from the previous solution, which will tend to reduce the number of iterations needed.

In [6]:
x2 = prox.do(x1, 1.0)

SCS using variable warm-starting
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 5.67e-02  1.01e+00  7.06e-02  1.07e+01  9.18e+00  0.00e+00  2.86e-03 
    20| 6.15e-06  1.54e-04  4.02e-06  1.07e+01  1.07e+01  3.30e-15  5.56e-03 
----------------------------------------------------------------------------
Status: Solved
Timing: Solve time: 5.58e-03s
	Lin-sys: nnz in L factor: 30408, avg solve time: 1.25e-04s
	Cones: avg projection time: 9.75e-07s
----------------------------------------------------------------------------
Error metrics:
dist(s, K) = 1.1102e-16, dist(y, K*) = 0.0000e+00, s'y/|s||y| = -2.5559e-16
|Ax + s - b|_2 / (1 + |b|_2) = 6.1523e-06
|A'y + c|_2 / (1 + |c|_2) = 1.5411e-04
|c'x + b'y| / (1 + |c'x| + |b'y|) = 4.0204e-06
---------------------------------------------------------------

Note that the `Prox.info` dict has been updated.

In [7]:
prox.info

{'iter': 20,
 'setup_time': 0.006726358,
 'solve_time': 0.0055819450000000005,
 'status': 'Solved'}

# Proximal iteration

As an example application, we can solve the original CVXPY problem through proximal iteration.
This involves repeated application of the prox operator.

In [8]:
for i in range(20):
    x0 = prox.do(x0, 1.0, verbose=False)

Note that, after several iterations, proximal iteration converges, and the SCS solver finishes in **0** iterations.

In [9]:
prox.info

{'iter': 0,
 'setup_time': 0.006726358,
 'solve_time': 0.000330299,
 'status': 'Solved'}

# Resetting warm-starting

We can also reset the internal warm-start vector to zero, by calling `Prox.reset_warm_start()`.

In [10]:
prox.do(x0)
prox.info

{'iter': 0,
 'setup_time': 0.006726358,
 'solve_time': 0.000431812,
 'status': 'Solved'}

Note that calling `Prox.reset_warm_start()` increases the number of SCS iterations required to find the solution.

In [11]:
prox.reset_warm_start()
prox.do(x0, verbose=False)
prox.info

{'iter': 20,
 'setup_time': 0.006726358,
 'solve_time': 0.004802309,
 'status': 'Solved'}

# Prox zero element

The `Prox` object is aware of its input variable names and sizes.
If we call `Prox.do()` without specifying `x0`, or setting it to `{}` or `None`,
the `Prox` object will automatically replace `x0` with the zero element of the
appropriate size, which the user can also access through `Prox.zero_elem`.

In [12]:
x0 = prox.do(verbose=True)

SCS using variable warm-starting
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 7.81e-02  9.28e-01  1.29e-02  1.26e+01  1.29e+01  0.00e+00  3.73e-03 
    20| 4.83e-05  9.95e-04  1.14e-05  1.11e+01  1.11e+01  1.65e-15  6.59e-03 
----------------------------------------------------------------------------
Status: Solved
Timing: Solve time: 6.61e-03s
	Lin-sys: nnz in L factor: 30408, avg solve time: 1.53e-03s
	Cones: avg projection time: 8.30e-06s
----------------------------------------------------------------------------
Error metrics:
dist(s, K) = 0.0000e+00, dist(y, K*) = 2.2204e-16, s'y/|s||y| = 1.1968e-16
|Ax + s - b|_2 / (1 + |b|_2) = 4.8319e-05
|A'y + c|_2 / (1 + |c|_2) = 9.9489e-04
|c'x + b'y| / (1 + |c'x| + |b'y|) = 1.1363e-05
----------------------------------------------------------------

In [13]:
prox.zero_elem

{'x': array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])}

# SCS settings

CySCS solver settings can be passed to the `Prox` object either during initialization or through `Prox.do()` as keyword arguments. We've already seen this with the `verbose=True` setting.

Some other common settings are to set the solver tolerance and the maximum number of iterations.

Settings set by the `Prox` constructor or `Prox.do()` persist until explicitly modified again.

In [14]:
x0 = prox.do(eps=1e-9, verbose=True)

SCS using variable warm-starting
----------------------------------------------------------------------------
 Iter | pri res | dua res | rel gap | pri obj | dua obj | kap/tau | time (s)
----------------------------------------------------------------------------
     0| 2.71e-05  7.55e-04  1.39e-05  1.11e+01  1.11e+01  0.00e+00  8.40e-03 
    60| 4.37e-13  1.54e-11  6.17e-13  1.11e+01  1.11e+01  1.53e-15  2.28e-02 
----------------------------------------------------------------------------
Status: Solved
Timing: Solve time: 2.28e-02s
	Lin-sys: nnz in L factor: 30408, avg solve time: 2.01e-04s
	Cones: avg projection time: 8.96e-07s
----------------------------------------------------------------------------
Error metrics:
dist(s, K) = 0.0000e+00, dist(y, K*) = 1.1102e-16, s'y/|s||y| = 1.0042e-16
|Ax + s - b|_2 / (1 + |b|_2) = 4.3750e-13
|A'y + c|_2 / (1 + |c|_2) = 1.5438e-11
|c'x + b'y| / (1 + |c'x| + |b'y|) = 6.1664e-13
----------------------------------------------------------------

In [15]:
prox.info

{'iter': 60,
 'setup_time': 0.006726358,
 'solve_time': 0.022789025,
 'status': 'Solved'}

# Problems with several variables

Many problems have more than one array variable for which we'd like to add proximal regularization.
These variables simply need to be named, and included in the `prox_vars` dict input to `Prox(prob, prox_vars)` constructor.

In [16]:
m, n = 10, 5

A = np.random.randn(m,n)
b = np.random.randn(m)

x = cvx.Variable(n)
y = cvx.Variable(m)
z = cvx.Variable()

obj = cvx.sum_squares(A*x-b) + cvx.norm(A.T*y - x) + 0.1*cvx.norm(y) + cvx.norm(z-y)

prob = cvx.Problem(cvx.Minimize(obj))

In [17]:
prox_vars = dict(x=x,y=y,z=z)
prox = Prox(prob, prox_vars)

Note that the output to `Prox.do()` is a dict with keys `'x'`, `'y'`, and `'z'`.

`'x'` and `'y'` correspond to `numpy.array` objects, while `'z'` is simply a Python `float`.

In [18]:
prox.do()

{'x': array([ 0.46352994,  0.22774606,  0.34405149, -0.4022498 , -0.28476353]),
 'y': array([ 0.00824547,  0.00818007,  0.00625243,  0.00743582,  0.0065997 ,
         0.00818434,  0.00755595,  0.00566383,  0.00944163,  0.01110097]),
 'z': 0.0078621414488371805}

# Many proxes in parallel

Since the underlying CySCS solver releases the Python GIL, we can evaluate many prox operators in parallel using multiple threads. Threads consume less memory than separate Python processes, and do not require data serialization to communicate between processes.

To see this in action, we'll first create several CVXPY problems and their prox operators.

In [19]:
m, n = 600, 400
k = 10
np.random.seed(0)

x = cvx.Variable(n)
xvars = {'x': x}

proxes = []
for _ in range(k):
    A = np.random.randn(m,n)
    b = np.random.randn(m)
    prob = cvx.Problem(cvx.Minimize(cvx.norm(A*x-b)))
    
    proxes += [Prox(prob,xvars, eps=1e-9)]

We'll use the Python `concurrent.futures` module for our parallel computation.
The module provides a `ThreadPoolExecutor` which has a `map()` method which distributes the computation across multiple threads.

`concurrent.futures` is part of the Python 3 standard library, and is backported to Python 2 via th

To use this interface, we define a simple function `do()` to map over our list of proxes, which does nothing more
than call the `Prox.do()` method. We would need only a slightly more complicated function to pass in specific `x0` values to each prox.

In [20]:
def do(prox):
    return prox.do()

We create a `ThreadPoolExecutor` with 2 threads, and ensure that the warm-start is reset.

In [21]:
from concurrent.futures import ThreadPoolExecutor

ex = ThreadPoolExecutor(2)

for prox in proxes:
    prox.reset_warm_start()

We can do the classic (non-parallel) computation by using the Python builtin `map()` function.

Note that this code is equivalent to the list comprehension

```python
output = [prox.do() for prox in proxes]
```

or the for loop

```python
output = []
for prox in proxes:
    output += [prox.do()]
```

We write it as a `map` here because its form is identical to the upcoming parallel computation.

We can time the cell to note the total wall clock time.

In [22]:
%%time
output = list(map(do, proxes))

CPU times: user 2.03 s, sys: 11.3 ms, total: 2.05 s
Wall time: 2.07 s


Making sure to reset the warm-start variables, we can do the computation in parallel by simply replacing
`map()` with `ex.map()`.

Note that the wall clock time is reduced, since we distribute the computation across two threads (on a dual-core machine).

In [23]:
for prox in proxes:
    prox.reset_warm_start()

In [24]:
%%time
output = list(ex.map(do, proxes))

CPU times: user 2.61 s, sys: 26.8 ms, total: 2.63 s
Wall time: 1.38 s


We can see that the sum of the SCS computation times is about twice the parallel wall clock time, which is exactly what we'd expect!

In [25]:
sum(prox.info['solve_time'] for prox in proxes)

2.650281737